# Alura Store · flujo de análisis

**Idea:** validar datos, calcular métricas por tienda y exportar resultados.

### 1) Setup

In [ ]:

from pathlib import Path
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

ROOT = Path.cwd().parents[1] if (Path.cwd().name == 'notebooks') else Path.cwd()
DATA, OUT, OUT_G = ROOT/'data', ROOT/'outputs', ROOT/'outputs'/'graficos'
OUT_G.mkdir(parents=True, exist_ok=True)
DATA, OUT, OUT_G


### 2) Unir CSV de `data/`

In [ ]:

frames = []
for p in sorted(DATA.glob("*.csv")):
    df = pd.read_csv(p)
    if "store" not in df.columns:
        df["store"] = p.stem.replace("_"," ").title()
    frames.append(df)
raw = pd.concat(frames, ignore_index=True)
raw.sample(min(5, len(raw)))


### 3) Ingresos por línea

In [ ]:

if "line_revenue" not in raw.columns:
    if {"quantity","unit_price"}.issubset(raw.columns):
        raw["line_revenue"] = raw["quantity"]*raw["unit_price"]
    elif "revenue" in raw.columns:
        raw["line_revenue"] = raw["revenue"]
    else:
        raw["line_revenue"] = 0.0
raw.head(3)


### 4) Resumen por tienda

In [ ]:

summary = (raw.groupby("store")["line_revenue"].sum()
           .rename("revenue_total").to_frame())

if "order_id" in raw.columns:
    summary = summary.join(raw.groupby("store")["order_id"].nunique().rename("unique_orders"))
    summary["rev_per_order"] = (summary["revenue_total"]/summary["unique_orders"]).round(2)

if "review_score" in raw.columns:
    summary = summary.join(raw.groupby("store")["review_score"].mean().rename("review_promedio").round(2))

if "shipping_days" in raw.columns:
    summary = summary.join(raw.groupby("store")["shipping_days"].mean().rename("envio_promedio_dias").round(2))

summary = summary.reset_index().sort_values("revenue_total", ascending=False)
summary


### 5) Gráficos rápidos

In [ ]:

plt.figure(); plt.bar(summary["store"], summary["revenue_total"])
plt.title("Ingresos por tienda"); plt.xticks(rotation=15); plt.tight_layout(); plt.show()

if "review_promedio" in summary.columns:
    plt.figure(); plt.bar(summary["store"], summary["review_promedio"])
    plt.title("Review promedio por tienda"); plt.ylim(0,5); plt.xticks(rotation=15); plt.tight_layout(); plt.show()

if "envio_promedio_dias" in summary.columns:
    plt.figure(); plt.bar(summary["store"], summary["envio_promedio_dias"])
    plt.title("Días de envío promedio"); plt.xticks(rotation=15); plt.tight_layout(); plt.show()


### 6) Ranking simple y recomendación

In [ ]:

def minmax(s):
    if s.max()==s.min(): return pd.Series(1.0, index=s.index)
    return (s - s.min())/(s.max()-s.min())

rank = summary.copy()
rank["costo_ingreso"] = 1 - minmax(rank["revenue_total"])
rank["costo_review"] = 1 - minmax(rank["review_promedio"]) if "review_promedio" in rank.columns else 0.5
rank["costo_envio"] = minmax(rank["envio_promedio_dias"]) if "envio_promedio_dias" in rank.columns else 0.5
w_ing,w_rev,w_ship = 0.45,0.35,0.20
rank["score_ineficiencia"] = w_ing*rank["costo_ingreso"] + w_rev*rank["costo_review"] + w_ship*rank["costo_envio"]
rank = rank.sort_values("score_ineficiencia", ascending=False).reset_index(drop=True)
rank


### 7) Exportar

In [ ]:

OUT.mkdir(parents=True, exist_ok=True)
(raw.groupby("store")["line_revenue"].sum()).to_csv(OUT/"ingresos_totales.csv")
summary.to_csv(OUT/"resumen_por_tienda.csv", index=False)
rank.to_csv(OUT/"ranking_ineficiencia.csv", index=False)

worst, best = rank.iloc[0], rank.iloc[-1]
reco = f"Recomendación: vender {worst['store']} (mejor: {best['store']})."
(OUT/"recomendacion_alura_store.txt").write_text(reco, encoding="utf-8")
reco
